# Aula prática - Integrando com a Assistant API

## Tópicos da aula

1. Acessando o Assistant API no DashBoard da plataforma da OpenAI.

2. Como editar o Assistant pela plataforma.

3. Criando um novo assistente passando um arquivo de dados (file_search).

4. Criando um novo assistente passando um arquivo de dados (code_interpreter)

In [ ]:
# Importando bibliotecas

from openai import OpenAI
import dotenv
import os


# Conectando com a API do OpenAI
dotenv.load_dotenv()

token = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=token)

## Criando um novo assistente utilizando um arquivo para realizar file_search

1. Primeiro devemos incluir um arquivo dentro da openai
2. Depois devemos criar um vector stores
4. Agora vamos criar o assistente

In [16]:
# Incluindo arquivo dentro da openai

file = open("dissertacao.pdf", "rb")
open_ai_file = client.files.create(file=file, purpose="assistants")

**vector_stores:** É um auxiliar para criar um armazenamento de vetores com file_ids e anexá-lo a este assistente. Pode haver no máximo 1 armazenamento de vetores anexado ao assistente.

In [17]:

# Criando um assistente com acesso ao vector store criado
my_assistant = client.beta.assistants.create(
    instructions="Você é um estudante de mestrado que completou sua tese e deve responder perguntas sobre ela. A tese se encontra no arquivo dissertacao.pdf.",
    name="Store owner",
    tools=[{"type": "file_search"}],
    tool_resources={"file_search": {"vector_stores": [{"file_ids": [open_ai_file.id]}]}},
    model="gpt-3.5-turbo"
)

## Testes com o assistente criado


Vamos usar o stream utilizado no tutorial para que o resultado apresentado pelo assistente apareça de forma iterativa.

1. Crie uma thread
2. Crie uma mensagem e a coloque na thread
2. Defina um EventHandler
3. Interaja com o assistente

In [19]:
# criando uma thread para o assistente
thread = client.beta.threads.create()

In [20]:
# criando uma mensagem
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="Quais os resultados obtidos na tese?"
)

In [7]:
from typing_extensions import override
from openai import AssistantEventHandler
 
 
class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print("\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print("\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)
 


In [21]:
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=my_assistant.id,
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > file_search


assistant > Os resultados obtidos na tese demonstraram que a inclusão das redes especialistas na pipeline de segmentação resultou em melhorias nos valores médios de coeficiente Dice (DSC) e Índice de Jaccard (IoU), bem como nas métricas de distância média e HD95. Isso evidenciou que a utilização das redes especialistas aprimorou a segmentação das estruturas propostas. Além disso, a votação majoritária se mostrou vantajosa, especialmente ao verificar as métricas de distâncias, destacando-se o HD95, demonstrando uma menor discrepância em relação à segmentação de referência. Portanto, a votação majoritária pode ser considerada vantajosa para melhorar a precisão na segmentação de imagens【4:0†source】.

A análise dos resultados das distâncias µSD e HD95 para as predições realizadas pelas redes generalistas mostrou que a arquitetura Unet obteve os melhores resultados médios para as estruturas avaliadas. Observou-se que a Unet teve um desempenho ligeiramente superior

## Criando um novo assistente utilizando um arquivo para realizar code_interpreter

1. Primeiro devemos incluir um arquivo dentro da openai
2. Depois devemos criar um vector stores
4. Agora vamos criar o assistente

In [30]:
# Incluindo arquivo dentro da openai

file = open("item_store.csv", "rb")
open_ai_file = client.files.create(file=file, purpose="assistants")

In [31]:
# Criando um assistente com acesso ao vector store criado
my_assistant = client.beta.assistants.create(
    instructions="Você é dono de uma loja que tem os itens listados no arquivo item_store.csv. Você deve responder perguntas sobre os itens da loja. O arquivo é do tipo csv.",
    name="Store owner",
    tools=[{"type": "code_interpreter"}],
    tool_resources={"code_interpreter": {"file_ids": [open_ai_file.id]}},
    model="gpt-3.5-turbo"
)

In [32]:
# criando uma thread para o assistente
thread = client.beta.threads.create()

# criando uma mensagem
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="Quais os vegetais disponíveis na loja?"
)

In [33]:
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=my_assistant.id,
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > code_interpreter

import pandas as pd

# Carregar o arquivo CSV
df = pd.read_csv('/mnt/data/file-ZmHqVTZQUQvuxyjxdcWvAyhm')

# Filtrar apenas os itens que são vegetais
vegetais = df[df['category'] == 'vegetable']['item'].unique()
vegetais
assistant > Parece que houve um erro ao tentar acessar a coluna 'category' no arquivo. Vou verificar a estrutura do arquivo para entender como podemos corrigir isso.# Verificar as primeiras linhas do arquivo para entender sua estrutura
df.head()
assistant > O arquivo tem as seguintes colunas: 'ItemID', 'Name', 'Category', 'Price' e 'Stock'. Vou corrigir a filtragem dos vegetais e listar os vegetais disponíveis na loja.# Filtrar apenas os itens que são vegetais
vegetais = df[df['Category'] == 'Vegetal']['Name'].unique()
vegetais
assistant > Os vegetais disponíveis na loja são: Brócolis e Cenoura.

In [34]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="Quais os preços desses vegetais"
)

In [35]:
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=my_assistant.id,
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > code_interpreter

# Filtrar os preços dos vegetais
precos_vegetais = df[df['Category'] == 'Vegetal'][['Name', 'Price']]
precos_vegetais
assistant > Os preços dos vegetais disponíveis na loja são:

- Brócolis: R$ 1,20
- Cenoura: R$ 0,90